<a href="https://colab.research.google.com/github/Hazakura2024/university-pytorch-DataAugmentation-fashionMNIST/blob/main/mnist_dataAugumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Python標準のライブラリ。存在しないキーにアクセスした際に自動で初期値を作成する辞書(dict)機能を提供
from collections import defaultdict

# グラフ描画ライブラリ。学習の進捗や結果を可視化するために使う
# as pltと書くことで、今後はpltという短い名前でこのライブラリの機能（例: plt.plot()）を呼び出せるように
import matplotlib.pyplot as plt

# 数値計算ライブラリ。多次元配列(行列)を効率的に扱うために使う
import numpy as np

# PyTorchの本体。Tensorの定義や基本的な計算機能を提供
import torch

# ニューラルネットワークの層(Linear, Conv2dなど)や活性化関数、損失関数などが含まれるモジュール
# nnの中には、全結合層 (nn.Linear) や畳み込み層 (nn.Conv2d)、活性化関数 (nn.ReLU) など、ネットワークの「層」を定義するためのクラスが多数含まれています。
import torch.nn as nn

# 画像認識用のデータセット(Fashion-MNISTなど)を簡単に利用するためのモジュール
# torchvisionは、画像認識の分野でPyTorchを助けるためのライブラリです。
# その中のdatasetsモジュールは、Fashion-MNISTやCIFAR10といった、
#  機械学習の練習や研究でよく使われるデータセットを簡単にダウンロードして利用する機能を提供します。
import torchvision.datasets as datasets

# 画像データに対する前処理(リサイズ、Tensorへの変換、Data Augmentationなど)を行う機能のモジュール
import torchvision.transforms as transforms

# ★ 変更点: Precision, Recall, F1値を計算するために scikit-learn をインポート
from sklearn.metrics import precision_recall_fscore_support



#Fashion-MNISTを読み込む

# 名前を簡単にしておく
# 画像データ（PILイメージやNumPy配列）を、PyTorchが計算で使えるTensor（テンソル）形式に変換します。
data_transform = transforms.ToTensor()


"""
ここに課題を作成する！！！
"""

train_transform = transforms.Compose([
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(10),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(
        # degrees=10,       # rotation_range=10 と同じ
    scale=(0.9, 1.1)  # zoom_range=0.1 (90%~110%) と同じ
    )
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 実験の条件を記載
augmenttationType = "scale"



"""
"""


# 学習データを読み込む DataLoader を作成する。

train_dataset = datasets.FashionMNIST(
    root = ".",
    train = True,
    download = True,
    transform = train_transform
    )
# root: データセットを保存するディレクトリのパスを指定します。
# データを保存する場所として、カレントディレクトリ（今いる場所）を示す "." を指定

# train: True を指定した場合は学習データ、False を指定した場合はテストデータをダウンロードします。

# transform: 画像に対して行う前処理をdata_transform = transforms.ToTensor()指定します。今回は、transforms.ToTensor のみを指定しました。 この Transformer は、画素値を範囲が [0, 255] の uint8 型から範囲が [0, 1] の float32 型にし、Pytorch で扱う Tensor に変換を行います
# 最低限、画像をTensorに変換する必要があるので transform=transforms.ToTensor() を指定

# download: True を指定した場合は、データセットがローカルにない場合は、ネットからダウンロードします。
# データがなければダウンロードしてほしいので download=True

# これでtrain_data_loaderは、訓練ループで使うと「シャッフルされた64枚ずつの画像とラベルのバッチ」を次々と提供してくれる、便利な「供給機」になります。
# Dataset（データのリスト）から、モデルが学習しやすいようにデータをまとめて取り出すための機能です。
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    # データを*4枚ずつの「束」（バッチ）**にして取り出すよう設
    batch_size=64,
    shuffle=True
)



# テストデータを読み込む DataLoader を作成する。

test_dataset = datasets.FashionMNIST(
    root = "datasets",
    train = False,
    transform = test_transform,
    download = True
)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=True
)








print(train_dataset)

'''
FashionMNIST クラスの以下の属性で、クラス一覧や各画像、ラベルを取得できます。

FashionMNIST.classes (list): クラス名一覧
FashionMNIST.class_to_idx (dict): クラス名、値がクラス ID の辞書
FashionMNIST.data (Tensor): 画像一覧
FashionMNIST.targets (Tensor): ラベル一覧

'''

# 各クラスのラベルを持つサンプルを1つずつ取得する。
class_ids, sample_indices = np.unique(train_dataset.targets, return_index=True);
# train_dataset.targets: 学習データセット内の全画像のラベル（0: T-shirt, 1: Trouser, ... 9: Ankle boot）が格納されているリスト

# np.unique(...): NumPyの関数で、重複する値を取り除いて、ユニークな値だけを返します。
# この場合、[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] という配列が class_ids に入ります。

# return_index=True: このオプションを付けると、ユニークな値が**初めて出現した位置（インデックス）**も一緒に返してくれます。
# 例えば、「ラベル0が最初に出てきたのはデータセットの何番目か」「ラベル1が最初に出てきたのは何番目か」…という情報が sample_indices に入ります。
# これにより、各クラスの代表サンプルを1つずつ効率的に見つけることができます。

fig = plt.figure(figsize=(10,4))
# これからグラフや画像を描画するための「キャンバス」や「台紙」のようなものを作成します。figsize=(10, 4) は、キャンバスのサイズを横10インチ、縦4インチに設定しています。

fig.suptitle("Example of every class in the Fashion-MNIST dataset だよ",fontsize="x-large")
# 作成したキャンバス全体に大きなタイトルを設定します。
# fontsize="x-large"で文字サイズを少し大きめにしています

for i in class_ids:
  img = train_dataset.data[sample_indices[i]]
  class_name = train_dataset.classes[i]

# キャンバスを2行5列のグリッドに分割し、(i+1)番目の領域を今回の描画エリア(ax)として設定する。
  ax = fig.add_subplot(2, 5, i + 1)

# 描画エリアに、クラスIDとクラス名をタイトルとして設定する。(例: "0: T-shirt/top")
  ax.set_title(f"{i}: {class_name}")

# 画像の枠線や目盛りを非表示にする。
  ax.set_axis_off()

# 取得した画像(img)をグレースケール(cmap="gray")で描画エリアに表示する。
  ax.imshow(img, cmap="gray")

# これまで設定した内容をすべて画面に表示する。
plt.show()




# --- ここからが追加するプレビューコード ---

print("--- Data Augmentation プレビュー ---")

# Augmentation（データ拡張）の変換処理だけを定義
# 注意: ToTensor() や Normalize() は、ここでは含めない
#      (画像として表示しにくくなるため)
augmentation_transform = transforms.Compose([
    # transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    # transforms.RandomRotation(10)
])

# 元となる画像データを1つ取得
# train_dataset.data[0] は (28, 28) のTensor
# 変換処理(transform)はPIL Imageを期待するため、ToPILImage()で変換
original_image_tensor = train_dataset.data[0]
original_pil_image = transforms.ToPILImage()(original_image_tensor)

# プレビューの数
num_previews = 5

# 描画用のキャンバスを作成 (元画像 + プレビュー5枚 = 合計6枚)
fig = plt.figure(figsize=(12, 3))
fig.suptitle(f"Data Augmentation ({augmenttationType}) のプレビュー", fontsize=14)
# fig.suptitle("Data Augmentationなしのプレビュー", fontsize=14)

# 1. 元画像をプロット
ax = fig.add_subplot(1, num_previews + 1, 1)
ax.set_title("Original")
ax.imshow(original_pil_image, cmap="gray")
ax.set_axis_off()

# 2. Data Augmentationを適用した画像をプロット
for i in range(num_previews):
    # 同じ元画像に、毎回ランダムな変換を適用
    transformed_image = augmentation_transform(original_pil_image)

    ax = fig.add_subplot(1, num_previews + 1, i + 2)
    ax.set_title(f"Preview {i + 1}")
    ax.imshow(transformed_image, cmap="gray")
    ax.set_axis_off()

# グラフを表示
plt.show()

# --- ここまでが追加するプレビューコード ---



SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-288381547.py, line 47)

In [ ]:
# Python標準のライブラリ。存在しないキーにアクセスした際に自動で初期値を作成する辞書(dict)機能を提供
from collections import defaultdict

# グラフ描画ライブラリ。学習の進捗や結果を可視化するために使う
# as pltと書くことで、今後はpltという短い名前でこのライブラリの機能（例: plt.plot()）を呼び出せるように
import matplotlib.pyplot as plt

# 数値計算ライブラリ。多次元配列(行列)を効率的に扱うために使う
import numpy as np

# PyTorchの本体。Tensorの定義や基本的な計算機能を提供
import torch

# ニューラルネットワークの層(Linear, Conv2dなど)や活性化関数、損失関数などが含まれるモジュール
# nnの中には、全結合層 (nn.Linear) や畳み込み層 (nn.Conv2d)、活性化関数 (nn.ReLU) など、ネットワークの「層」を定義するためのクラスが多数含まれています。
import torch.nn as nn

# 画像認識用のデータセット(Fashion-MNISTなど)を簡単に利用するためのモジュール
# torchvisionは、画像認識の分野でPyTorchを助けるためのライブラリです。
# その中のdatasetsモジュールは、Fashion-MNISTやCIFAR10といった、
#  機械学習の練習や研究でよく使われるデータセットを簡単にダウンロードして利用する機能を提供します。
import torchvision.datasets as datasets

# 画像データに対する前処理(リサイズ、Tensorへの変換、Data Augmentationなど)を行う機能のモジュール
import torchvision.transforms as transforms

# ★ 変更点: Precision, Recall, F1値を計算するために scikit-learn をインポート
from sklearn.metrics import precision_recall_fscore_support



#Fashion-MNISTを読み込む

# 名前を簡単にしておく
# 画像データ（PILイメージやNumPy配列）を、PyTorchが計算で使えるTensor（テンソル）形式に変換します。
data_transform = transforms.ToTensor()


"""
ここに課題を作成する！！！
"""

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(10),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(
            degrees=10,       # rotation_range=10 と同じ
            scale=(0.9, 1.1)  # zoom_range=0.1 (90%~110%) と同じ
        ),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 実験の条件を記載
augmenttationType = "Web"



"""
"""


# 学習データを読み込む DataLoader を作成する。

train_dataset = datasets.FashionMNIST(
    root = ".",
    train = True,
    download = True,
    transform = train_transform
    )
# root: データセットを保存するディレクトリのパスを指定します。
# データを保存する場所として、カレントディレクトリ（今いる場所）を示す "." を指定

# train: True を指定した場合は学習データ、False を指定した場合はテストデータをダウンロードします。

# transform: 画像に対して行う前処理をdata_transform = transforms.ToTensor()指定します。今回は、transforms.ToTensor のみを指定しました。 この Transformer は、画素値を範囲が [0, 255] の uint8 型から範囲が [0, 1] の float32 型にし、Pytorch で扱う Tensor に変換を行います
# 最低限、画像をTensorに変換する必要があるので transform=transforms.ToTensor() を指定

# download: True を指定した場合は、データセットがローカルにない場合は、ネットからダウンロードします。
# データがなければダウンロードしてほしいので download=True

# これでtrain_data_loaderは、訓練ループで使うと「シャッフルされた64枚ずつの画像とラベルのバッチ」を次々と提供してくれる、便利な「供給機」になります。
# Dataset（データのリスト）から、モデルが学習しやすいようにデータをまとめて取り出すための機能です。
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    # データを*4枚ずつの「束」（バッチ）**にして取り出すよう設
    batch_size=64,
    shuffle=True
)



# テストデータを読み込む DataLoader を作成する。

test_dataset = datasets.FashionMNIST(
    root = "datasets",
    train = False,
    transform = test_transform,
    download = True
)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=True
)








print(train_dataset)

'''
FashionMNIST クラスの以下の属性で、クラス一覧や各画像、ラベルを取得できます。

FashionMNIST.classes (list): クラス名一覧
FashionMNIST.class_to_idx (dict): クラス名、値がクラス ID の辞書
FashionMNIST.data (Tensor): 画像一覧
FashionMNIST.targets (Tensor): ラベル一覧

'''

# 各クラスのラベルを持つサンプルを1つずつ取得する。
class_ids, sample_indices = np.unique(train_dataset.targets, return_index=True);
# train_dataset.targets: 学習データセット内の全画像のラベル（0: T-shirt, 1: Trouser, ... 9: Ankle boot）が格納されているリスト

# np.unique(...): NumPyの関数で、重複する値を取り除いて、ユニークな値だけを返します。
# この場合、[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] という配列が class_ids に入ります。

# return_index=True: このオプションを付けると、ユニークな値が**初めて出現した位置（インデックス）**も一緒に返してくれます。
# 例えば、「ラベル0が最初に出てきたのはデータセットの何番目か」「ラベル1が最初に出てきたのは何番目か」…という情報が sample_indices に入ります。
# これにより、各クラスの代表サンプルを1つずつ効率的に見つけることができます。

fig = plt.figure(figsize=(10,4))
# これからグラフや画像を描画するための「キャンバス」や「台紙」のようなものを作成します。figsize=(10, 4) は、キャンバスのサイズを横10インチ、縦4インチに設定しています。

fig.suptitle("Example of every class in the Fashion-MNIST dataset だよ",fontsize="x-large")
# 作成したキャンバス全体に大きなタイトルを設定します。
# fontsize="x-large"で文字サイズを少し大きめにしています

for i in class_ids:
  img = train_dataset.data[sample_indices[i]]
  class_name = train_dataset.classes[i]

# キャンバスを2行5列のグリッドに分割し、(i+1)番目の領域を今回の描画エリア(ax)として設定する。
  ax = fig.add_subplot(2, 5, i + 1)

# 描画エリアに、クラスIDとクラス名をタイトルとして設定する。(例: "0: T-shirt/top")
  ax.set_title(f"{i}: {class_name}")

# 画像の枠線や目盛りを非表示にする。
  ax.set_axis_off()

# 取得した画像(img)をグレースケール(cmap="gray")で描画エリアに表示する。
  ax.imshow(img, cmap="gray")

# これまで設定した内容をすべて画面に表示する。
plt.show()




# --- ここからが追加するプレビューコード ---

print("--- Data Augmentation プレビュー ---")

# Augmentation（データ拡張）の変換処理だけを定義
# 注意: ToTensor() や Normalize() は、ここでは含めない
#      (画像として表示しにくくなるため)
augmentation_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(10)
    # transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(
        degrees=10,       # rotation_range=10 と同じ
        scale=(0.9, 1.1)  # zoom_range=0.1 (90%~110%) と同じ
    )
])

# 元となる画像データを1つ取得
# train_dataset.data[0] は (28, 28) のTensor
# 変換処理(transform)はPIL Imageを期待するため、ToPILImage()で変換
original_image_tensor = train_dataset.data[0]
original_pil_image = transforms.ToPILImage()(original_image_tensor)

# プレビューの数
num_previews = 5

# 描画用のキャンバスを作成 (元画像 + プレビュー5枚 = 合計6枚)
fig = plt.figure(figsize=(12, 3))
fig.suptitle(f"Data Augmentation ({augmenttationType}) のプレビュー", fontsize=14)
# fig.suptitle("Data Augmentationなしのプレビュー", fontsize=14)

# 1. 元画像をプロット
ax = fig.add_subplot(1, num_previews + 1, 1)
ax.set_title("Original")
ax.imshow(original_pil_image, cmap="gray")
ax.set_axis_off()

# 2. Data Augmentationを適用した画像をプロット
for i in range(num_previews):
    # 同じ元画像に、毎回ランダムな変換を適用
    transformed_image = augmentation_transform(original_pil_image)

    ax = fig.add_subplot(1, num_previews + 1, i + 2)
    ax.set_title(f"Preview {i + 1}")
    ax.imshow(transformed_image, cmap="gray")
    ax.set_axis_off()

# グラフを表示
plt.show()

# --- ここまでが追加するプレビューコード ---



'''
「画像の特徴を自動で見つけ出すのが得意な、人間の視覚に似た仕組みを持つネットワーク」

CNNモデルとは、「畳み込みニューラルネットワーク（Convolutional Neural Network）」の略で、主に画像認識の分野で圧倒的な成果を上げているディープラーニングモデルの一種です 🧠。
ひと言でいうと、「画像の特徴を自動で見つけ出すのが得意な、人間の視覚に似た仕組みを持つネットワーク」です。

CNNは何がすごいのか？
従来のAIにとって、画像（例: 猫の写真）を認識するのは非常に困難でした。AIにとって画像は「ただの数字（ピクセル）の集まり」であり、猫が画像の右にいても、左にいても、大きくても、小さくても「同じ猫」だと理解するのが難しかったのです。

CNNは、この問題を「畳み込み層」と「プーリング層」という特別な仕組みで解決しました。

畳み込み層 (Convolutional Layer) 🔍

「カーネル（フィルター）」と呼ばれる小さな"虫眼鏡"を使って、画像全体をスキャンします。
この"虫眼鏡"は、「縦線」「横線」「特定の模様」など、画像の中の単純な特徴を探します。
層が深くなるにつれて、単純な特徴（線）が組み合わさり、「目」や「耳」といった複雑な部品を検出できるようになります。

プーリング層 (Pooling Layer) 圧縮

畳み込み層が見つけた特徴マップを「圧縮」します。
重要な特徴だけを残し、画像のサイズを小さくすることで、計算を効率化します。
また、画像の中で特徴の位置が少しズレても、同じ特徴として認識できるようにする役割もあります（これがCNNの強力な点です）。

CNNモデルとは、「畳み込みニューラルネットワーク（Convolutional Neural Network）」の略で、主に画像認識の分野で圧倒的な成果を上げているディープラーニングモデルの一種です 🧠。

ひと言でいうと、「画像の特徴を自動で見つけ出すのが得意な、人間の視覚に似た仕組みを持つネットワーク」です。

CNNは何がすごいのか？
従来のAIにとって、画像（例: 猫の写真）を認識するのは非常に困難でした。AIにとって画像は「ただの数字（ピクセル）の集まり」であり、猫が画像の右にいても、左にいても、大きくても、小さくても「同じ猫」だと理解するのが難しかったのです。

CNNは、この問題を「畳み込み層」と「プーリング層」という特別な仕組みで解決しました。

CNNの主な仕組み
CNNは、大きく分けて2つの部分で構成されています。

1. 特徴抽出部（画像から特徴を見つける）
この部分で、「畳み込み層」と「プーリング層」が活躍します。

畳み込み層 (Convolutional Layer) 🔍

「カーネル（フィルター）」と呼ばれる小さな"虫眼鏡"を使って、画像全体をスキャンします。
この"虫眼鏡"は、「縦線」「横線」「特定の模様」など、画像の中の単純な特徴を探します。
層が深くなるにつれて、単純な特徴（線）が組み合わさり、「目」や「耳」といった複雑な部品を検出できるようになります。

プーリング層 (Pooling Layer) 圧縮

畳み込み層が見つけた特徴マップを「圧縮」します。
重要な特徴だけを残し、画像のサイズを小さくすることで、計算を効率化します。
また、画像の中で特徴の位置が少しズレても、同じ特徴として認識できるようにする役割もあります（これがCNNの強力な点です）。


2. 分類部（特徴を元に判断する）
全結合層 (Fully Connected Layer) 🧠
特徴抽出部が見つけ出した「目」「耳」「ひげ」といった様々な特徴の情報をすべて受け取ります。
これらの特徴を総合的に判断し、「これだけ"猫"の特徴が揃っているから、この画像は99%の確率で"猫"だろう」と**最終的な分類（予測）**を行います。
'''

# CNN モデルを作成する

# PyTorchのすべてのニューラルネットワークモデルの「親」である nn.Module を継承（引き継いで）、
# Net という名前の新しいクラス（設計図）を作成します。
class Net(nn.Module):

  #コンストラクタと呼ばれるメソッドです。
  # model = Net() のように、このクラス（設計図）から実際のモデル（モノ）を作るときに一度だけ呼び出されます。
  def __init__(self):

    super().__init__()

    # nn.Sequentialは、複数の層をひとまとめにする「容器」です。
    # ここでは、画像から特徴を抽出する部分（畳み込み層とプーリング層）をまとめて self.features という名前を付けています。
    self.features = nn.Sequential(
        # 2D畳み込み層です。
        # 1: 入力チャンネル数（Fashion-MNISTは白黒画像なので1）
        # 32: 出力チャンネル数（32種類の特徴を検出する）
        # kernel_size=3: 3x3のフィルターで画像の特徴をスキャンします。
        # padding=1: 画像のフチに1ピクセルの余白を追加します。これにより、3x3のフィルターを使っても画像のサイズ（28x28）が変わりません。
        nn.Conv2d(1, 32, kernel_size=3, padding=1),

        #**活性化関数（ReLU）**です。
        # Conv2dで計算した結果のうち、マイナスになった値を0に置き換えます。これにより、ネットワークがより複雑なパターンを学習できるようになります。
        nn.ReLU(),

        #マックスプーリング層です。
        # kernel_size=2: 2x2の領域を見て、その中で一番大きい値だけを残します。
        # 役割: 画像のサイズを半分に圧縮します（例: 28x28 → 14x14）。これにより、重要な特徴だけを残し、計算量を減らします。
        nn.MaxPool2d(kernel_size=2),

        # 2層目の2D畳み込み層です。
        # 32: 1層目の出力チャンネル数（32）と合わせる必要があります。
        # 64: さらに多くの（64種類の）特徴を検出します。
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.ReLU(),


        # 再びプーリングを行い、画像サイズをさらに半分にします（例: 14x14 → 7x7）。
        nn.MaxPool2d(kernel_size=2)
    )

    # 特徴抽出が終わった後の、最終的な「分類」を行う部分（全結合層）をまとめて self.classifier という名前を付けています。
    self.classifier = nn.Sequential(

        # ドロップアウト層です。
        # 役割: 過学習（学習データにだけ詳しくなりすぎること）を防ぐためのテクニックです。
        # 学習中、ランダムに一部のニューロンを「一時的に無効化」し、モデルが特定のパターンに依存しすぎるのを防ぎます。
        nn.Dropout(),

        # **全結合層（または線形層）**です。
        # self.features の最後で、画像は 64 チャンネル、7x7 のサイズになっています。
        # 全結合層に入力するには、この3Dデータ（64x7x7 = 3136）を1列のベクトルに「平坦化（flatten）」する必要があります。この 3136 が入力の数です。
        nn.Linear(64 * 7 * 7, 128),
        nn.ReLU(),
        nn.Dropout(),

        # 最後の全結合層です。
        # 128: 前の層の出力（128）と合わせます。
        # 10: 最終的な出力の数。Fashion-MNISTは10クラス（Tシャツ、ズボン…など）に分類するため、必ず10になります。
        nn.Linear(128,10),

        # 最後の層の出力を「対数確率」に変換します。
        # 10個の出力（スコア）を、合計が1になる確率に変換（Softmax）し、さらにその対数（Log）を取ります。これは、後で使う損失関数（NLLLoss）とセットで使われることが多いです。
        nn.LogSoftmax(dim=1),
    )

  # **フォワード（順伝播）**メソッドです。
  # __init__で定義した部品（層）を使って、データが実際に流れる順序を定義します。
  # xは入力データ（画像のバッチ）です。
  def forward(self,x):

    # 入力データxを、self.features（Conv, ReLU, Pool, Conv, ReLU, Pool）にまとめて通します。
    # データの形状変化: [バッチサイズ, 1, 28, 28] → [バッチサイズ, 64, 7, 7]
    x = self.features(x)

    # データを「平坦化」します。
    # self.classifier（全結合層）に入れるために、[バッチサイズ, 64, 7, 7]のデータを[バッチサイズ, 3136]の形状に変換します。
    # start_dim=1 は「0番目の次元（バッチサイズ）はそのまま残し、1番目以降の次元（64, 7, 7）をすべて平坦化する」という意味です。
    x = torch.flatten(x,1)

    # 平坦化されたデータxを、self.classifier（Dropout, Linear, ReLU, Dropout, Linear, LogSoftmax）にまとめて通します。
    # データの形状変化: [バッチサイズ, 3136] → [バッチサイズ, 10]
    x = self.classifier(x)

    # 最終的な計算結果（10クラス分の対数確率）を返します。
    return x




# 損失関数を作成する。
# 「モデルの予測と、実際の正解との『ズレ』を計算する関数」のことです。
nll_loss = nn.NLLLoss(reduction='sum')



# 計算を実行するデバイスを選択する。
# CUDA が利用可能な場合は、GPU、そうでない場合は CPU を選択する
# pythonの三項演算子 A if 条件 else B
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# モデルを計算を実行するデバイスに転送する。
model = Net().to(device)


# 最適化手法を選択
optim = torch.optim.Adam(model.parameters())


'''
モデルの**重み（Weight）**とは、モデルが予測を行うために使う、無数の「**調整ノブ**」のようなものです。

そして「**重みパラメータの更新**」とは、モデルの予測が正解に近づくように、この**「調整ノブ」を少しずつ回して調整していく作業**のことを指します。

これは、機械学習モデルの「**学習**」そのものを意味する、最も重要なプロセスです。

---

### なぜ更新が必要なの？

* **最初はデタラメ**: 新しく作ったモデルの「重み（調整ノブ）」は、最初はランダムな値（デタラメな設定）になっています。そのため、当然ながら予測もデタラメです。
* **正解に近づけたい**: このデタラメな予測と、実際の「正解」を比べます。その「**ズレ（＝損失）**」を計算します。
* **修正する**: その「ズレ」をできるだけ小さくするために、「こっちのノブは少し右に、あっちのノブは少し左に回そう」と、すべてのノブの設定を微調整します。これが「更新」です。



---

### ラジオのチューニングに例えると 📻

「重みの更新」は、古いラジオで放送局の電波（正解）に合わせる作業によく似ています。

1.  **初期状態 (重みがランダム)**:
    ラジオのチューニングノブ（＝重み）が適当な位置にあるため、ザーザーというノイズ（＝デタラメな予測）しか聞こえません。

2.  **損失の計算 (ズレの確認)**:
    「もっとクリアな音が聞きたい（正解）」と「今のノイズ（予測）」の**差**を感じ取ります。これが「損失」です。

3.  **勾配の計算 (どっちに回すべきか？)**:
    ノブを**右に回したら**ノイズが少しマシになったか？ **左に回したら**マシになったか？を調べます。これが「勾配（どっちの方向に進めば良くなるか）」の計算です。
    (PyTorchでは `loss.backward()` がこれを自動でやってくれます)

4.  **重みの更新 (ノブを回す)**:
    「右に回した方が良さそうだ」と分かったら、ノブを**そっと右に回します**。
    (PyTorchでは `optimizer.step()` がこの調整を実行します)

この「**音を聞く → ズレを確認 → 回す方向を決める → そっと回す**」という作業を何度も何度も繰り返すことで、ノブ（重み）は最適な位置に調整され、電波（正解）をクリアに受信（予測）できるようになります。

これがモデルの学習、すなわち「重みパラメータの更新」の正体です。
'''




# 学習を行う関数を作成
# model（ニューラルネットワーク）、device（GPU/CPU）、data_loader（データ供給機）、optim（最適化アルゴリズム）の4つを引数として受け取ります。
def train(model, device, data_loader, optim):
  """
    1エポック分の学習を行う。
        :param model: モデル
        :param device: デバイス
        :param data_loader: Data Loader
        :param optim: Optimizer
  """
  # modelを学習モードに設定する。
  model.train()

  # このエポック（データセット1周）で発生した損失（ズレ）の合計値を記録するための変数
  total_loss = 0

  # このエポックで正解した予測の合計数を記録するための変数
  total_correct = 0

  # ★ 変更点: エポック全体の予測と正解ラベルを保存するリスト
  all_targets = []
  all_preds = []

  # データ及びラベルを計算を実行するデバイスに転送する。
  # data_loader（データ供給機）から、データをバッチ（束）単位で取り出すループを開始します。
  # data には画像データ（例: 64枚分）が、target にはそれに対応する正解ラベル（例: 64個分）が入ります。
  for data, target in data_loader:
    data, target = data.to(device), target.to(device)

    # 順伝搬を行う
    # モデルに画像データ data を入力し、予測結果 output を計算させます。（forwardメソッドが実行されます）
    output = model(data)

    # 損失関数の値を計算する
    # モデルの予測 output と、正解の target を nll_loss（損失関数）に渡し、どれだけ「ズレ」ているかを計算します。この loss の値が小さいほど、予測が正しかったことを意味します。
    loss = nll_loss(output, target)
    # float(loss)（または .item()）は、PyTorchのTensorからPythonの数値だけを取り出すおまじないで、メモリ効率のために重要です。
    total_loss += float(loss)




    # 逆伝搬を行う
    # **勾配の初期化（リセット）**です。PyTorchは勾配を自動で「加算」してしまうため、このバッチの計算を始める前に、必ず前のバッチで計算した勾配をリセット（0に）します。これを忘れると学習が正しく進みません。
    optim.zero_grad()
    # **逆伝搬（Backward Pass）**です。
    # 計算された loss（ズレ）を最小化するために、モデル内の全パラメータ（重み）をそれぞれどちらの方向にどれだけ調整すべきか（＝勾配）を自動で計算します。
    loss.backward()

    # パラメータを更新する
    # optim（最適化アルゴリズム）が、loss.backward()で計算された勾配を元に、モデルの全パラメータを**実際に更新（調整）**します。
    # これが「学習」の瞬間です。
    optim.step()




    # 確率の最も高いクラスを予測ラベルとする。
    # モデルの output（各クラスの確率）から、最も確率が高いクラスのインデックスを予測結果 pred_target として取り出します。
    pred_target = output.argmax(dim=1)

    # 正答数を計算する
    total_correct += int((pred_target == target).sum())

    # ★ 変更点: 予測と正解ラベルをリストに溜め込む (計算用にCPUに戻す)
    all_targets.extend(target.cpu().numpy())
    all_preds.extend(pred_target.cpu().numpy())


  # 損失関数の値の平均及び精度を計算する。
  # 予測 pred_target と正解 target を比較し、True（正解）の数を合計（.sum()）して total_correct に加算します。
  avg_loss = total_loss / len(data_loader.dataset)
  # avg_loss = total_loss / len(data_loader.dataset)
  accuracy = total_correct / len(data_loader.dataset)

  # ★ 変更点: エポック全体の予測と正解を使ってメトリクスを計算
    # average='macro': 全クラスの平均を計算 (Fahion-MNISTは10クラスあるため)
    # zero_division=0: 警告（0除算）を非表示にする
  precision, recall, f1, _ = precision_recall_fscore_support(
      all_targets, all_preds, average='macro', zero_division=0
  )

  return avg_loss, accuracy, precision, recall, f1


"""
ここまで地道にやったけど、いったん完成を目指してコピペ
"""




# 評価を行う関数を作成する

def test(model, device, data_loader):
    """
    テストデータに対する損失の平均及び精度を計算する。
        :param model: モデル
        :param device: デバイス
        :param data_loader: Data Loader
    """
    # モデルをテストモードに設定する。
    model.eval()

# ★ 変更点: エポック全体の予測と正解ラベルを保存するリスト
    all_targets = []
    all_preds = []

    with torch.no_grad():
        total_loss = 0
        total_correct = 0
        for data, target in data_loader:
            # データ及びラベルを計算を実行するデバイスに転送する。
            data, target = data.to(device), target.to(device)

            # 順伝搬する。
            output = model(data)

            # 損失を計算する。
            loss = nll_loss(output, target)
            total_loss += float(loss)

            # 確率の最も高いクラスを予測ラベルとする。
            pred_target = output.argmax(dim=1)

            # 正答数を計算する。
            total_correct += int((pred_target == target).sum())

    # ★ 変更点: 予測と正解ラベルをリストに溜め込む (計算用にCPUに戻す)
            all_targets.extend(target.cpu().numpy())
            all_preds.extend(pred_target.cpu().numpy())

    # 損失の平均及び精度を計算する。
    avg_loss = total_loss / len(data_loader.dataset)
    accuracy = total_correct / len(data_loader.dataset)

    # ★ 変更点: エポック全体の予測と正解を使ってメトリクスを計算
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_targets, all_preds, average='macro', zero_division=0
    )

    # ★ 変更点: 4つの値を返す
    return avg_loss, accuracy, precision, recall, f1


# 学習する
n_epochs = 100

history = defaultdict(list)

print('csv形式で表示')
print(f"epoch,{augmenttationType} [train] loss,{augmenttationType} [train] accuracy,{augmenttationType} [train] precision,{augmenttationType} [train] recall,{augmenttationType} [train] f1,{augmenttationType} [test] loss,{augmenttationType} [test] accuracy,{augmenttationType} [test] precision,{augmenttationType} [test] recall,{augmenttationType} [test] f1")

for epoch in range(n_epochs):
    # ★ 変更点: 戻り値が5つになった
    train_loss, train_accuracy, train_precision, train_recall, train_f1 = train(
        model, device, train_data_loader, optim
    )
    history["train_loss"].append(train_loss)
    history["train_accuracy"].append(train_accuracy)
    history["train_precision"].append(train_precision)
    history["train_recall"].append(train_recall)
    history["train_f1"].append(train_f1)

    # ★ 変更点: 戻り値が5つになった
    test_loss, test_accuracy, test_precision, test_recall, test_f1 = test(
        model, device, test_data_loader
    )
    history["test_loss"].append(test_loss)
    history["test_accuracy"].append(test_accuracy)
    history["test_precision"].append(test_precision)
    history["test_recall"].append(test_recall)
    history["test_f1"].append(test_f1)



    # ★ 変更点: print文を更新
    print(
        f"epoch {epoch + 1},"
        f"{train_loss:.6f},{train_accuracy:.4f},"
        f"{train_precision:.4f},{train_recall:.4f},{train_f1:.4f},"
        f"{test_loss:.6f},{test_accuracy:.4f},"
        f"{test_precision:.4f},{test_recall:.4f},{test_f1:.4f}"
    )



# 損失関数の値の推移をグラフ化する。

epochs = np.arange(1, n_epochs + 1)

# ★ 変更点: グラフを3つ (1行3列) にする
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=(15, 4))

# 損失の推移
ax1.set_title("Loss")
ax1.plot(epochs, history["train_loss"], label="train")
ax1.plot(epochs, history["test_loss"], label="test")
ax1.set_xlabel("Epoch")
ax1.legend()

# 精度の推移
ax2.set_title("Accuracy")
ax2.plot(epochs, history["train_accuracy"], label="train")
ax2.plot(epochs, history["test_accuracy"], label="test")
ax2.set_xlabel("Epoch")
ax2.legend()

# ★ 変更点: F1, Precision, Recall の推移
ax3.set_title("F1 / Precision / Recall")
ax3.plot(epochs, history["test_f1"], label="test F1-Score")
ax3.plot(epochs, history["test_precision"], label="test Precision")
ax3.plot(epochs, history["test_recall"], label="test Recall")
ax3.set_xlabel("Epoch")
ax3.legend()

# グラフをぴったり合わせる
plt.tight_layout()
plt.show()

KeyboardInterrupt: 

# Python標準のライブラリ。存在しないキーにアクセスした際に自動で初期値を作成する辞書(dict)機能を提供
from collections import defaultdict

# グラフ描画ライブラリ。学習の進捗や結果を可視化するために使う
# as pltと書くことで、今後はpltという短い名前でこのライブラリの機能（例: plt.plot()）を呼び出せるように
import matplotlib.pyplot as plt

# 数値計算ライブラリ。多次元配列(行列)を効率的に扱うために使う
import numpy as np

# PyTorchの本体。Tensorの定義や基本的な計算機能を提供
import torch

# ニューラルネットワークの層(Linear, Conv2dなど)や活性化関数、損失関数などが含まれるモジュール
# nnの中には、全結合層 (nn.Linear) や畳み込み層 (nn.Conv2d)、活性化関数 (nn.ReLU) など、ネットワークの「層」を定義するためのクラスが多数含まれています。
import torch.nn as nn

# 画像認識用のデータセット(Fashion-MNISTなど)を簡単に利用するためのモジュール
# torchvisionは、画像認識の分野でPyTorchを助けるためのライブラリです。
# その中のdatasetsモジュールは、Fashion-MNISTやCIFAR10といった、
#  機械学習の練習や研究でよく使われるデータセットを簡単にダウンロードして利用する機能を提供します。
import torchvision.datasets as datasets

# 画像データに対する前処理(リサイズ、Tensorへの変換、Data Augmentationなど)を行う機能のモジュール
import torchvision.transforms as transforms



#Fashion-MNISTを読み込む

# 名前を簡単にしておく
# 画像データ（PILイメージやNumPy配列）を、PyTorchが計算で使えるTensor（テンソル）形式に変換します。
data_transform = transforms.ToTensor()


"""
ここに課題を作成する！！！
"""

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])




"""
"""


# 学習データを読み込む DataLoader を作成する。

train_dataset = datasets.FashionMNIST(
    root = ".",
    train = True,
    download = True,
    transform = train_transform
    )
# root: データセットを保存するディレクトリのパスを指定します。
# データを保存する場所として、カレントディレクトリ（今いる場所）を示す "." を指定

# train: True を指定した場合は学習データ、False を指定した場合はテストデータをダウンロードします。

# transform: 画像に対して行う前処理をdata_transform = transforms.ToTensor()指定します。今回は、transforms.ToTensor のみを指定しました。 この Transformer は、画素値を範囲が [0, 255] の uint8 型から範囲が [0, 1] の float32 型にし、Pytorch で扱う Tensor に変換を行います
# 最低限、画像をTensorに変換する必要があるので transform=transforms.ToTensor() を指定

# download: True を指定した場合は、データセットがローカルにない場合は、ネットからダウンロードします。
# データがなければダウンロードしてほしいので download=True

# これでtrain_data_loaderは、訓練ループで使うと「シャッフルされた64枚ずつの画像とラベルのバッチ」を次々と提供してくれる、便利な「供給機」になります。
# Dataset（データのリスト）から、モデルが学習しやすいようにデータをまとめて取り出すための機能です。
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    # データを*4枚ずつの「束」（バッチ）**にして取り出すよう設
    batch_size=64,
    shuffle=True
)



# テストデータを読み込む DataLoader を作成する。

test_dataset = datasets.FashionMNIST(
    root = "datasets",
    train = False,
    transform = test_transform,
    download = True
)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=True
)








print(train_dataset)

'''
FashionMNIST クラスの以下の属性で、クラス一覧や各画像、ラベルを取得できます。

FashionMNIST.classes (list): クラス名一覧
FashionMNIST.class_to_idx (dict): クラス名、値がクラス ID の辞書
FashionMNIST.data (Tensor): 画像一覧
FashionMNIST.targets (Tensor): ラベル一覧

'''

# 各クラスのラベルを持つサンプルを1つずつ取得する。
class_ids, sample_indices = np.unique(train_dataset.targets, return_index=True);
# train_dataset.targets: 学習データセット内の全画像のラベル（0: T-shirt, 1: Trouser, ... 9: Ankle boot）が格納されているリスト

# np.unique(...): NumPyの関数で、重複する値を取り除いて、ユニークな値だけを返します。
# この場合、[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] という配列が class_ids に入ります。

# return_index=True: このオプションを付けると、ユニークな値が**初めて出現した位置（インデックス）**も一緒に返してくれます。
# 例えば、「ラベル0が最初に出てきたのはデータセットの何番目か」「ラベル1が最初に出てきたのは何番目か」…という情報が sample_indices に入ります。
# これにより、各クラスの代表サンプルを1つずつ効率的に見つけることができます。

fig = plt.figure(figsize=(10,4))
# これからグラフや画像を描画するための「キャンバス」や「台紙」のようなものを作成します。figsize=(10, 4) は、キャンバスのサイズを横10インチ、縦4インチに設定しています。

fig.suptitle("Example of every class in the Fashion-MNIST dataset だよ",fontsize="x-large")
# 作成したキャンバス全体に大きなタイトルを設定します。
# fontsize="x-large"で文字サイズを少し大きめにしています

for i in class_ids:
  img = train_dataset.data[sample_indices[i]]
  class_name = train_dataset.classes[i]

# キャンバスを2行5列のグリッドに分割し、(i+1)番目の領域を今回の描画エリア(ax)として設定する。
  ax = fig.add_subplot(2, 5, i + 1)

# 描画エリアに、クラスIDとクラス名をタイトルとして設定する。(例: "0: T-shirt/top")
  ax.set_title(f"{i}: {class_name}")

# 画像の枠線や目盛りを非表示にする。
  ax.set_axis_off()

# 取得した画像(img)をグレースケール(cmap="gray")で描画エリアに表示する。
  ax.imshow(img, cmap="gray")

# これまで設定した内容をすべて画面に表示する。
plt.show()


'''
「画像の特徴を自動で見つけ出すのが得意な、人間の視覚に似た仕組みを持つネットワーク」

CNNモデルとは、「畳み込みニューラルネットワーク（Convolutional Neural Network）」の略で、主に画像認識の分野で圧倒的な成果を上げているディープラーニングモデルの一種です 🧠。
ひと言でいうと、「画像の特徴を自動で見つけ出すのが得意な、人間の視覚に似た仕組みを持つネットワーク」です。

CNNは何がすごいのか？
従来のAIにとって、画像（例: 猫の写真）を認識するのは非常に困難でした。AIにとって画像は「ただの数字（ピクセル）の集まり」であり、猫が画像の右にいても、左にいても、大きくても、小さくても「同じ猫」だと理解するのが難しかったのです。

CNNは、この問題を「畳み込み層」と「プーリング層」という特別な仕組みで解決しました。

畳み込み層 (Convolutional Layer) 🔍

「カーネル（フィルター）」と呼ばれる小さな"虫眼鏡"を使って、画像全体をスキャンします。
この"虫眼鏡"は、「縦線」「横線」「特定の模様」など、画像の中の単純な特徴を探します。
層が深くなるにつれて、単純な特徴（線）が組み合わさり、「目」や「耳」といった複雑な部品を検出できるようになります。

プーリング層 (Pooling Layer) 圧縮

畳み込み層が見つけた特徴マップを「圧縮」します。
重要な特徴だけを残し、画像のサイズを小さくすることで、計算を効率化します。
また、画像の中で特徴の位置が少しズレても、同じ特徴として認識できるようにする役割もあります（これがCNNの強力な点です）。

CNNモデルとは、「畳み込みニューラルネットワーク（Convolutional Neural Network）」の略で、主に画像認識の分野で圧倒的な成果を上げているディープラーニングモデルの一種です 🧠。

ひと言でいうと、「画像の特徴を自動で見つけ出すのが得意な、人間の視覚に似た仕組みを持つネットワーク」です。

CNNは何がすごいのか？
従来のAIにとって、画像（例: 猫の写真）を認識するのは非常に困難でした。AIにとって画像は「ただの数字（ピクセル）の集まり」であり、猫が画像の右にいても、左にいても、大きくても、小さくても「同じ猫」だと理解するのが難しかったのです。

CNNは、この問題を「畳み込み層」と「プーリング層」という特別な仕組みで解決しました。

CNNの主な仕組み
CNNは、大きく分けて2つの部分で構成されています。

1. 特徴抽出部（画像から特徴を見つける）
この部分で、「畳み込み層」と「プーリング層」が活躍します。

畳み込み層 (Convolutional Layer) 🔍

「カーネル（フィルター）」と呼ばれる小さな"虫眼鏡"を使って、画像全体をスキャンします。
この"虫眼鏡"は、「縦線」「横線」「特定の模様」など、画像の中の単純な特徴を探します。
層が深くなるにつれて、単純な特徴（線）が組み合わさり、「目」や「耳」といった複雑な部品を検出できるようになります。

プーリング層 (Pooling Layer) 圧縮

畳み込み層が見つけた特徴マップを「圧縮」します。
重要な特徴だけを残し、画像のサイズを小さくすることで、計算を効率化します。
また、画像の中で特徴の位置が少しズレても、同じ特徴として認識できるようにする役割もあります（これがCNNの強力な点です）。


2. 分類部（特徴を元に判断する）
全結合層 (Fully Connected Layer) 🧠
特徴抽出部が見つけ出した「目」「耳」「ひげ」といった様々な特徴の情報をすべて受け取ります。
これらの特徴を総合的に判断し、「これだけ"猫"の特徴が揃っているから、この画像は99%の確率で"猫"だろう」と**最終的な分類（予測）**を行います。
'''

# CNN モデルを作成する

# PyTorchのすべてのニューラルネットワークモデルの「親」である nn.Module を継承（引き継いで）、
# Net という名前の新しいクラス（設計図）を作成します。
class Net(nn.Module):

  #コンストラクタと呼ばれるメソッドです。
  # model = Net() のように、このクラス（設計図）から実際のモデル（モノ）を作るときに一度だけ呼び出されます。
  def __init__(self):

    super().__init__()

    # nn.Sequentialは、複数の層をひとまとめにする「容器」です。
    # ここでは、画像から特徴を抽出する部分（畳み込み層とプーリング層）をまとめて self.features という名前を付けています。
    self.features = nn.Sequential(
        # 2D畳み込み層です。
        # 1: 入力チャンネル数（Fashion-MNISTは白黒画像なので1）
        # 32: 出力チャンネル数（32種類の特徴を検出する）
        # kernel_size=3: 3x3のフィルターで画像の特徴をスキャンします。
        # padding=1: 画像のフチに1ピクセルの余白を追加します。これにより、3x3のフィルターを使っても画像のサイズ（28x28）が変わりません。
        nn.Conv2d(1, 32, kernel_size=3, padding=1),

        #**活性化関数（ReLU）**です。
        # Conv2dで計算した結果のうち、マイナスになった値を0に置き換えます。これにより、ネットワークがより複雑なパターンを学習できるようになります。
        nn.ReLU(),

        #マックスプーリング層です。
        # kernel_size=2: 2x2の領域を見て、その中で一番大きい値だけを残します。
        # 役割: 画像のサイズを半分に圧縮します（例: 28x28 → 14x14）。これにより、重要な特徴だけを残し、計算量を減らします。
        nn.MaxPool2d(kernel_size=2),

        # 2層目の2D畳み込み層です。
        # 32: 1層目の出力チャンネル数（32）と合わせる必要があります。
        # 64: さらに多くの（64種類の）特徴を検出します。
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.ReLU(),


        # 再びプーリングを行い、画像サイズをさらに半分にします（例: 14x14 → 7x7）。
        nn.MaxPool2d(kernel_size=2)
    )

    # 特徴抽出が終わった後の、最終的な「分類」を行う部分（全結合層）をまとめて self.classifier という名前を付けています。
    self.classifier = nn.Sequential(

        # ドロップアウト層です。
        # 役割: 過学習（学習データにだけ詳しくなりすぎること）を防ぐためのテクニックです。
        # 学習中、ランダムに一部のニューロンを「一時的に無効化」し、モデルが特定のパターンに依存しすぎるのを防ぎます。
        nn.Dropout(),

        # **全結合層（または線形層）**です。
        # self.features の最後で、画像は 64 チャンネル、7x7 のサイズになっています。
        # 全結合層に入力するには、この3Dデータ（64x7x7 = 3136）を1列のベクトルに「平坦化（flatten）」する必要があります。この 3136 が入力の数です。
        nn.Linear(64 * 7 * 7, 128),
        nn.ReLU(),
        nn.Dropout(),

        # 最後の全結合層です。
        # 128: 前の層の出力（128）と合わせます。
        # 10: 最終的な出力の数。Fashion-MNISTは10クラス（Tシャツ、ズボン…など）に分類するため、必ず10になります。
        nn.Linear(128,10),

        # 最後の層の出力を「対数確率」に変換します。
        # 10個の出力（スコア）を、合計が1になる確率に変換（Softmax）し、さらにその対数（Log）を取ります。これは、後で使う損失関数（NLLLoss）とセットで使われることが多いです。
        nn.LogSoftmax(dim=1),
    )

  # **フォワード（順伝播）**メソッドです。
  # __init__で定義した部品（層）を使って、データが実際に流れる順序を定義します。
  # xは入力データ（画像のバッチ）です。
  def forward(self,x):

    # 入力データxを、self.features（Conv, ReLU, Pool, Conv, ReLU, Pool）にまとめて通します。
    # データの形状変化: [バッチサイズ, 1, 28, 28] → [バッチサイズ, 64, 7, 7]
    x = self.features(x)

    # データを「平坦化」します。
    # self.classifier（全結合層）に入れるために、[バッチサイズ, 64, 7, 7]のデータを[バッチサイズ, 3136]の形状に変換します。
    # start_dim=1 は「0番目の次元（バッチサイズ）はそのまま残し、1番目以降の次元（64, 7, 7）をすべて平坦化する」という意味です。
    x = torch.flatten(x,1)

    # 平坦化されたデータxを、self.classifier（Dropout, Linear, ReLU, Dropout, Linear, LogSoftmax）にまとめて通します。
    # データの形状変化: [バッチサイズ, 3136] → [バッチサイズ, 10]
    x = self.classifier(x)

    # 最終的な計算結果（10クラス分の対数確率）を返します。
    return x




# 損失関数を作成する。
# 「モデルの予測と、実際の正解との『ズレ』を計算する関数」のことです。
nll_loss = nn.NLLLoss()



# 計算を実行するデバイスを選択する。
# CUDA が利用可能な場合は、GPU、そうでない場合は CPU を選択する
# pythonの三項演算子 A if 条件 else B
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# モデルを計算を実行するデバイスに転送する。
model = Net().to(device)


# 最適化手法を選択
optim = torch.optim.Adam(model.parameters())


'''
モデルの**重み（Weight）**とは、モデルが予測を行うために使う、無数の「**調整ノブ**」のようなものです。

そして「**重みパラメータの更新**」とは、モデルの予測が正解に近づくように、この**「調整ノブ」を少しずつ回して調整していく作業**のことを指します。

これは、機械学習モデルの「**学習**」そのものを意味する、最も重要なプロセスです。

---

### なぜ更新が必要なの？

* **最初はデタラメ**: 新しく作ったモデルの「重み（調整ノブ）」は、最初はランダムな値（デタラメな設定）になっています。そのため、当然ながら予測もデタラメです。
* **正解に近づけたい**: このデタラメな予測と、実際の「正解」を比べます。その「**ズレ（＝損失）**」を計算します。
* **修正する**: その「ズレ」をできるだけ小さくするために、「こっちのノブは少し右に、あっちのノブは少し左に回そう」と、すべてのノブの設定を微調整します。これが「更新」です。



---

### ラジオのチューニングに例えると 📻

「重みの更新」は、古いラジオで放送局の電波（正解）に合わせる作業によく似ています。

1.  **初期状態 (重みがランダム)**:
    ラジオのチューニングノブ（＝重み）が適当な位置にあるため、ザーザーというノイズ（＝デタラメな予測）しか聞こえません。

2.  **損失の計算 (ズレの確認)**:
    「もっとクリアな音が聞きたい（正解）」と「今のノイズ（予測）」の**差**を感じ取ります。これが「損失」です。

3.  **勾配の計算 (どっちに回すべきか？)**:
    ノブを**右に回したら**ノイズが少しマシになったか？ **左に回したら**マシになったか？を調べます。これが「勾配（どっちの方向に進めば良くなるか）」の計算です。
    (PyTorchでは `loss.backward()` がこれを自動でやってくれます)

4.  **重みの更新 (ノブを回す)**:
    「右に回した方が良さそうだ」と分かったら、ノブを**そっと右に回します**。
    (PyTorchでは `optimizer.step()` がこの調整を実行します)

この「**音を聞く → ズレを確認 → 回す方向を決める → そっと回す**」という作業を何度も何度も繰り返すことで、ノブ（重み）は最適な位置に調整され、電波（正解）をクリアに受信（予測）できるようになります。

これがモデルの学習、すなわち「重みパラメータの更新」の正体です。
'''




# 学習を行う関数を作成
# model（ニューラルネットワーク）、device（GPU/CPU）、data_loader（データ供給機）、optim（最適化アルゴリズム）の4つを引数として受け取ります。
def train(model, device, data_loader, optim):
  """
    1エポック分の学習を行う。
        :param model: モデル
        :param device: デバイス
        :param data_loader: Data Loader
        :param optim: Optimizer
  """
  # modelを学習モードに設定する。
  model.train()

  # このエポック（データセット1周）で発生した損失（ズレ）の合計値を記録するための変数
  total_loss = 0

  # このエポックで正解した予測の合計数を記録するための変数
  total_correct = 0

  # データ及びラベルを計算を実行するデバイスに転送する。
  # data_loader（データ供給機）から、データをバッチ（束）単位で取り出すループを開始します。
  # data には画像データ（例: 64枚分）が、target にはそれに対応する正解ラベル（例: 64個分）が入ります。
  for data, target in data_loader:
    data, target = data.to(device), target.to(device)

    # 順伝搬を行う
    # モデルに画像データ data を入力し、予測結果 output を計算させます。（forwardメソッドが実行されます）
    output = model(data)

    # 損失関数の値を計算する
    # モデルの予測 output と、正解の target を nll_loss（損失関数）に渡し、どれだけ「ズレ」ているかを計算します。この loss の値が小さいほど、予測が正しかったことを意味します。
    loss = nll_loss(output, target)
    # float(loss)（または .item()）は、PyTorchのTensorからPythonの数値だけを取り出すおまじないで、メモリ効率のために重要です。
    total_loss += float(loss)




    # 逆伝搬を行う
    # **勾配の初期化（リセット）**です。PyTorchは勾配を自動で「加算」してしまうため、このバッチの計算を始める前に、必ず前のバッチで計算した勾配をリセット（0に）します。これを忘れると学習が正しく進みません。
    optim.zero_grad()
    # **逆伝搬（Backward Pass）**です。
    # 計算された loss（ズレ）を最小化するために、モデル内の全パラメータ（重み）をそれぞれどちらの方向にどれだけ調整すべきか（＝勾配）を自動で計算します。
    loss.backward()

    # パラメータを更新する
    # optim（最適化アルゴリズム）が、loss.backward()で計算された勾配を元に、モデルの全パラメータを**実際に更新（調整）**します。
    # これが「学習」の瞬間です。
    optim.step()




    # 確率の最も高いクラスを予測ラベルとする。
    # モデルの output（各クラスの確率）から、最も確率が高いクラスのインデックスを予測結果 pred_target として取り出します。
    pred_target = output.argmax(dim=1)

    # 正答数を計算する
    total_correct += int((pred_target == target).sum())

  # 損失関数の値の平均及び精度を計算する。
  # 予測 pred_target と正解 target を比較し、True（正解）の数を合計（.sum()）して total_correct に加算します。
  avg_loss = total_loss / len(data_loader.dataset)
  # avg_loss = total_loss / len(data_loader.dataset)
  accuracy = total_correct / len(data_loader.dataset)

  return avg_loss, accuracy


"""
ここまで地道にやったけど、いったん完成を目指してコピペ
"""




# 評価を行う関数を作成する

def test(model, device, data_loader):
    """
    テストデータに対する損失の平均及び精度を計算する。
        :param model: モデル
        :param device: デバイス
        :param data_loader: Data Loader
    """
    # モデルをテストモードに設定する。
    model.eval()

    with torch.no_grad():
        total_loss = 0
        total_correct = 0
        for data, target in data_loader:
            # データ及びラベルを計算を実行するデバイスに転送する。
            data, target = data.to(device), target.to(device)

            # 順伝搬する。
            output = model(data)

            # 損失を計算する。
            loss = nll_loss(output, target)
            total_loss += float(loss)

            # 確率の最も高いクラスを予測ラベルとする。
            pred_target = output.argmax(dim=1)

            # 正答数を計算する。
            total_correct += int((pred_target == target).sum())

    # 損失の平均及び精度を計算する。
    avg_loss = total_loss / len(data_loader.dataset)
    accuracy = total_correct / len(data_loader.dataset)

    return avg_loss, accuracy







# 学習する
n_epochs = 50

history = defaultdict(list)
for epoch in range(n_epochs):
    # 1エポック分、学習する。
    train_loss, train_accuracy = train(model, device, train_data_loader, optim)
    history["train_loss"].append(train_loss)
    history["train_accuracy"].append(train_accuracy)

    # 評価する。
    test_loss, test_accuracy = test(model, device, test_data_loader)
    history["test_loss"].append(test_loss)
    history["test_accuracy"].append(test_accuracy)

    print(
        f"epoch {epoch + 1} "
        f"[train] loss: {train_loss:.6f}, accuracy: {train_accuracy:.0%} "
        f"[test] loss: {test_loss:.6f}, accuracy: {test_accuracy:.0%}"
    )




# 損失関数の値の推移をグラフ化する。

epochs = np.arange(1, n_epochs + 1)

fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(8, 3))

# 損失の推移
ax1.set_title("Loss")
ax1.plot(epochs, history["train_loss"], label="train")
ax1.plot(epochs, history["test_loss"], label="test")
ax1.set_xlabel("Epoch")
ax1.legend()

# 精度の推移
ax2.set_title("Accuracy")
ax2.plot(epochs, history["train_accuracy"], label="train")
ax2.plot(epochs, history["test_accuracy"], label="test")
ax2.set_xlabel("Epoch")
ax2.legend()

plt.show()
